In [ ]:
#global env python 3.11.7

import numpy as np
import pandas as pd
import pingouin as pg
import pickle
from scipy.stats import ttest_ind, mannwhitneyu

In [104]:
merged_df = pd.read_pickle("fusion/merged_df.p")

In [105]:
df_ger = pd.read_pickle("fusion/df_ger.p")
df_usa = pd.read_pickle("fusion/df_usa.p")

In [106]:
continuous_vars = [
 #'year',
 #'recipient_status',
 'recipient_age',
 'recipient_bmi',
 #'recipient_sex',
 #'recipient_blood_type',
 'MELD',
 'cold_ischemia_time',
 'donor_age',
 'donor_bmi',
 #'donor_sex',
 #'donor_blood_type',
 #'donor_drugs',
 #'donor_alcohol',
 'donor_alt',
 'donor_ast',
 'donor_bili',
 'donor_sodium',
 #'1yr_graft',
 #'country',
]

In [107]:
cols_remaining = merged_df.columns.difference(continuous_vars)
categorical_vars = cols_remaining.tolist()

categorical_vars.remove('country')

In [108]:
results = []

# Continuous variables -> t test
for col in continuous_vars:
    ttest = pg.ttest(df_ger[col], df_usa[col], correction=False)
    results.append({
        'variable': col,
        'test': 't-test',
        'p-value': ttest['p-val'].iloc[0],
        'CI95%': ttest['CI95%'].iloc[0],
        'cohen-d / Cramer': ttest['cohen-d'].iloc[0],
        'chi2': '/',
    })

In [109]:
# Categorical variables -> chi2 test
for col in categorical_vars:
    chi2 = pg.chi2_independence(merged_df, x='country', y=col)
    chi2_test = chi2[2]
    results.append({
        'variable': col,
        'test': chi2_test['test'].iloc[0],
        'p-value': chi2_test['pval'].iloc[0],
        'CI95%': '/',
        'cohen-d / Cramer': chi2_test['cramer'].iloc[0],
        'chi2': chi2_test['chi2'].iloc[0],
    })

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pingouin/contingency.py:151: UserWarning: Low count on observed frequencies.
  warnings.warn(f"Low count on {name} frequencies.")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_stats_py.py:8073: RuntimeWarning: divide by zero encountered in power
  terms = f_obs * ((f_obs / f_exp)**lambda_ - 1)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_stats_py.py:8073: RuntimeWarning: invalid value encountered in multiply
  terms = f_obs * ((f_obs / f_exp)**lambda_ - 1)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_stats_py.py:8070: RuntimeWarning: divide by zero encountered in divide
  terms = 2.0 * special.xlogy(f_exp, f_exp / f_obs)


In [110]:
results_df = pd.DataFrame(results)

In [111]:
results_df

,variable,test,p-value,CI95%,cohen-d / Cramer,chi2
0,recipient_age,t-test,3.834046e-12,"[-1.1, -0.61]",0.076238,/
1,recipient_bmi,t-test,0.000000e+00,"[-2.61, -2.37]",0.435624,/
2,MELD,t-test,8.243714e-08,"[-0.84, -0.39]",0.059474,/
3,cold_ischemia_time,t-test,0.000000e+00,"[204.68, 216.16]",1.290543,/
4,donor_age,t-test,0.000000e+00,"[12.35, 13.05]",0.777947,/
5,donor_bmi,t-test,1.870664e-96,"[-1.6, -1.32]",0.229051,/
6,donor_alt,t-test,3.739894e-05,"[-10.97, -3.9]",0.050706,/
7,donor_ast,t-test,1.777356e-12,"[8.37, 14.82]",0.087854,/
8,donor_bili,t-test,7.047682e-58,"[-3.69, -2.88]",0.210022,/
9,donor_sodium,t-test,5.170982e-147,"[-2.63, -2.26]",0.297860,/


In [112]:
results_df.to_excel('fusion/pingouin.xlsx')